# Attributes to Extract
Bold = Completed
- **Requisites and Anti-requisites**
- **Credit points**
- **Result type**
- **Amount of groupwork**
- **Type of assessments**
- **Content (topics)**
- **Description**

In [1]:
import json

In [2]:
def jsonToDict(filePath):
    with open(filePath) as json_file: 
        return json.load(json_file)

In [3]:
def saveAsJSON(filePath, data):
    with open(filePath, 'w') as fp:
        json.dump(data, fp, indent = 4)

In [4]:
subjectData = jsonToDict("./data/subjectPages.json")

In [5]:
subjectDetails = {}

In [6]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [7]:
def strAfterTerminationStr(text, indexStart, terminationStr, splitStr=None):
    textAfter = ""
    charIndex = indexStart
    textLength = len(text) - len(terminationStr)

    while text[charIndex:charIndex+len(terminationStr)] != terminationStr and charIndex < textLength:
        textAfter += text[charIndex]
        charIndex += 1

    if splitStr == None:
        return textAfter.strip()
    else:
        subStrs = textAfter.strip().split(splitStr)

        for i, subStr in enumerate(subStrs):
            if not hasNumbers(subStr):
                subStrs.remove(subStr)
            else:
                subStrs[i] = subStr.strip()

        return subStrs

In [8]:
def getTextAfterSubStr(text, subStr, numChars=None, terminationStr=None, splitStr=None):
    indexStart = text.find(subStr)+len(subStr)
    
    if numChars != None:
        return text[indexStart:indexStart+numChars].strip()
    
    if terminationStr != None:
        return strAfterTerminationStr(text, indexStart, terminationStr, splitStr)

In [9]:
def find_all(a_string, sub):
    result = []
    k = 0
    
    while k < len(a_string):
        k = a_string.find(sub, k)
        if k == -1:
            return result
        else:
            result.append(k)
            k += 1 #change to k += len(sub) to not search overlapping results
            
    return result

In [10]:
for subjectCode in subjectData:
    details = {}
    subjectPage = subjectData[subjectCode]
    
    details["credit_points"] = getTextAfterSubStr(subjectPage, 
                                                  "<em>Credit points:</em>", 2)
    details["requisites"] = getTextAfterSubStr(subjectPage, "Requisite(s):", 
                                               terminationStr="</em>", 
                                               splitStr="AND")
    details["anti_requisites"] = getTextAfterSubStr(subjectPage, "Anti-requisite(s):", 
                                                   terminationStr="</em>", splitStr="AND")
    details["result_type"] = getTextAfterSubStr(subjectPage, "Result type</em>:", 
                                                terminationStr="<br/>")
    details["description"] = getTextAfterSubStr(subjectPage, "<h3>Description</h3>\n<p>", 
                                                terminationStr="</p>")
    details["content_topics"] = getTextAfterSubStr(subjectPage, '<h3>Content (topics)</h3>\n<p>', 
                                                   terminationStr="</p>")
    details["num_group_assessments"] = subjectPage.count("<td>Group")

    assessment_types = set([])
    
    for strPos in find_all(subjectPage, '<th class="assessmentTaskTableMainHeading">Type:</th>'):
        assessment_type = strAfterTerminationStr(subjectPage, 
                                                 strPos+len('<th class="assessmentTaskTableMainHeading">Type:</th><td>'), 
                                                 "</td>")[1:]
        
        if assessment_type not in assessment_types:
            assessment_types.add(assessment_type)
    
    details["assessment_types"] = list(assessment_types)
    
    subjectDetails[subjectCode] = details

In [11]:
for subjectCode in subjectDetails:
    for requisite in subjectDetails[subjectCode]["requisites"]:
        if requisite[0:4] == "tent" or requisite[0:len("><!-- Content Columns -->")] == "><!-- Content Columns -->":
            subjectDetails[subjectCode]["requisites"].remove(requisite)

In [12]:
for subjectCode in subjectDetails:
    for anti_requisite in subjectDetails[subjectCode]["anti_requisites"]:
        if anti_requisite[0:4] == "tent" or anti_requisite[0:len("><!-- Content Columns -->")] == "><!-- Content Columns -->":
            subjectDetails[subjectCode]["anti_requisites"].remove(anti_requisite)

In [13]:
subjectDetails["31250"]["requisites"]

[]

In [14]:
subjectDetails["48240"]["requisites"]

['<a href="https://handbook.uts.edu.au/subjects/33130.html">33130</a> Mathematical Modelling 1',
 '<a href="https://handbook.uts.edu.au/subjects/48230.html">48230</a> Engineering Communication']

In [15]:
saveAsJSON("./data/majorsSubjectDetails.json", subjectDetails)

## Working out why the number of subjects wasn't matching depending on the JSON file I looked at

In [16]:
subjectNames = jsonToDict("./data/deprecated/subjectData.json")

for subjectCode in subjectNames:
    if subjectCode in subjectDetails:
        subjectDetails[subjectCode]["name"] = subjectNames[subjectCode]["name"]

In [17]:
numNotFound = 0
notFound = set([])

for subjectCode in subjectDetails:
    if "name" not in subjectDetails[subjectCode]:
        notFound.add(subjectCode)
        numNotFound += 1
        
print(notFound)
print(numNotFound)

{'22567', '22566', '79015', '23418', '79036', '24223', '22573', '22240', '97806', '23570', '42050', '24510', '97805', '97804', '22575', '25500', '24207', '24306', '23022', '79603', '23507', '97803', '79018', '97807', '79011', '59330', '22574', '24211', '23021', '24224', '59333', '79606', '23999', '22576', '70110', '21999', '21506', '22017', '24205', '20100', '79013', '79032', '24101', '21595', '21591', '23005', '24212', '79033', '23569', '97802', '97640', '23504', '24104', '22016', '97801', '24999', '97808'}
57


In [18]:
len(subjectDetails)

465

In [19]:
len(jsonToDict("./data/deprecated/subjectData.json"))

967

In [20]:
len(subjectData)

465

In [21]:
# degreeSubjects = jsonToDict("./data/degreeSubjects.json")
# subjectTypes = ("core", "major", "sub-major", "stream")
# foundSubjects = set([])

# for degreeCode in degreeSubjects:
#     for subjectType in subjectTypes:
#         if subjectType == "core":
#             for subjectCode in degreeSubjects[degreeCode]["subjects"][subjectType]:
#                     foundSubjects.add(subjectCode)
#         else:
#             for subjectCode in degreeSubjects[degreeCode]["subjects"][subjectType]["core"]:
#                 foundSubjects.add(subjectCode)

#             for subjectCode in degreeSubjects[degreeCode]["subjects"][subjectType]["elective"]:
#                 foundSubjects.add(subjectCode)
            
# print(len(foundSubjects))

## Further data cleaning and collation

In [32]:
degreeMajorSubject = jsonToDict("./data/degreeMajorSubject.json")
majorData = jsonToDict("./data/deprecated/majorData.json")
# print(degreeMajorSubject)

subjectTypes = ("major", "sub-major", "stream")
print(subjectTypes)

for degreeCode in degreeMajorSubject:
    for subjectType in subjectTypes:
        del degreeMajorSubject[degreeCode]["subjects"][subjectType]
        
        pluralSubjectType = f"{subjectType}s"
        degreeMajorSubject[degreeCode]["subjects"][pluralSubjectType] = {}
        
for majorCode in majorData:
    degreeCode = majorData[majorCode]["degreeCode"]
    
    for subjectType in subjectTypes:
        pluralSubjectType = f"{subjectType}s"
        degreeMajorSubject[degreeCode]["subjects"][pluralSubjectType][majorCode] = majorData[majorCode] 
        
print(degreeMajorSubject)

saveAsJSON("./data/degreeMajorSubject.json", degreeMajorSubject)

('major', 'sub-major', 'stream')
{'c10026': {'name': 'Bachelor of Business', 'subjects': {'core': {'26100': {'name': 'Integrating Business Perspectives'}, '22107': {'name': 'Accounting for Business Decisions A'}, '23115': {'name': 'Economics for Business'}, '26134': {'name': 'Business Statistics'}, '22207': {'name': 'Accounting for Business Decisions B'}, '21129': {'name': 'Managing People and Organisations'}, '24108': {'name': 'Marketing Foundations'}, '25300': {'name': 'Fundamentals of Business Finance'}, '22420': {'name': 'Accounting Standards and Regulations'}, '22321': {'name': 'Cost Management Systems'}, '22320': {'name': 'Accounting for Business Combinations'}, '22522': {'name': 'Assurance Services and Audit'}, '79017': {'name': 'Taxation Law'}, '22421': {'name': 'Management Decisions and Control'}, '22319': {'name': 'Financial Statement Analysis (Capstone)'}, '21036': {'name': 'Managing Strategic Performance'}, '21407': {'name': 'Strategic Human Resource Management'}, '21037': 